In this script we study the ligands obtained in "AllBinding.ipynb" in order to optimize the data and create the traning set.

1. With RDKit we obtain some features of the NRas ligands: 
    - Molecular weight 
    - H-bond donor
    - H-bond acceptor
    - TPSA
    - Rotable bonds

2. We discard the molecules that are repeated (with the smiles column).

In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, Crippen

df = pd.read_csv('../Data/NRAS_ligands.csv')
print(df.columns)

Index(['BindingDB Reactant_set_id', 'Ligand SMILES', 'Ligand InChI',
       'Ligand InChI Key', 'BindingDB MonomerID', 'BindingDB Ligand Name',
       'Target Name',
       'Target Source Organism According to Curator or DataSource',
       'IC50 (nM)'],
      dtype='object')


In [2]:
features = []

for i, row in df.iterrows():
    estructura = row['Ligand SMILES']
    
    mol = Chem.MolFromSmiles(estructura)
    
    if mol is not None:
        # Amb les funcions RDKit extreim les dades 
        smiles = row['Ligand SMILES']
        formula = Chem.rdMolDescriptors.CalcMolFormula(mol)
        mw = Descriptors.ExactMolWt(mol)
        logp = Crippen.MolLogP(mol)
        num_hbd = Descriptors.NumHDonors(mol)
        num_hba = Descriptors.NumHAcceptors(mol)
        tpsa = Descriptors.TPSA(mol)
        num_rb = Descriptors.NumRotatableBonds(mol)
        ic50 = row['IC50 (nM)']
        
        # Agregam les dades a una llista
        features.append([smiles, formula, mw, logp, num_hbd, num_hba, tpsa, num_rb, ic50])

# cream un data frame amb les dades extretes
df_features = pd.DataFrame(features, columns=['SMILES','Formula', 'Molecular weight', 'LogP', "H-bond donor","H-bond acceptor","TPSA","Rotatable bonds", "IC50"])
print(len(df_features))
print(df_features.head(5))

[06:35:36] Explicit valence for atom # 27 N, 4, is greater than permitted
[06:35:38] Explicit valence for atom # 27 N, 4, is greater than permitted
[06:35:39] Explicit valence for atom # 27 N, 4, is greater than permitted
[06:35:41] Explicit valence for atom # 27 N, 4, is greater than permitted


1831
                                              SMILES     Formula  \
0    COc1cc2ncc(-c3cccc(NC4CCNC4)n3)n2cc1-c1cn[nH]c1   C20H21N7O   
1      COc1cc2ncc(-c3cccc(NC4CCNC4)n3)n2cc1-c1cccnc1   C22H22N6O   
2      COc1cc2ncc(-c3cccc(NC4CCNC4)n3)n2cc1-c1ccncc1   C22H22N6O   
3  Cc1n[nH]c(C)c1-c1cn2c(cnc2cc1CO)-c1cccc(NC2CCN...   C22H25N7O   
4  COc1cc2ncc(-c3cccc(NC4CCNC4)n3)n2cc1-c1cnn(CCN...  C26H32N8O2   

   Molecular weight     LogP  H-bond donor  H-bond acceptor    TPSA  \
0        375.180758  2.56880             3                7   92.16   
1        386.185509  3.24070             2                7   76.37   
2        386.185509  3.24070             2                7   76.37   
3        403.212058  2.66934             4                7  103.16   
4        488.264822  2.37440             2               10   93.77   

   Rotatable bonds    IC50  
0                5   33170  
1                5    9920  
2                5    1830  
3                5   15720  
4             

necessito treure dades que hi ha del ic50 que tenen simbols de menor i major 

In [3]:
df_features = df_features[df_features['IC50'].str.contains(">|<")== False]
summary = df_features.describe()
print(summary)
print(df_features)

#ara si que ho paso a numeric 
nombre_columna = 'IC50'
df_features[nombre_columna] = pd.to_numeric(df_features[nombre_columna], errors='coerce')

       Molecular weight         LogP  H-bond donor  H-bond acceptor  \
count       1443.000000  1443.000000   1443.000000      1443.000000   
mean         929.880380     5.904186      2.243243        11.722800   
std          111.243312     0.926376      0.499390         1.349762   
min          299.093773     0.510500      2.000000         5.000000   
25%          913.485045     5.544500      2.000000        11.000000   
50%          937.488416     5.996000      2.000000        12.000000   
75%          977.519717     6.401700      2.000000        13.000000   
max         1154.630408     8.069200      6.000000        15.000000   

              TPSA  Rotatable bonds  
count  1443.000000      1443.000000  
mean    168.349473         9.802495  
std      19.032505         1.439658  
min      54.250000         3.000000  
25%     161.890000         9.000000  
50%     171.540000        10.000000  
75%     177.740000        11.000000  
max     206.800000        14.000000  
                  

In [4]:
summary = df_features.describe()
print(summary)

       Molecular weight         LogP  H-bond donor  H-bond acceptor  \
count       1443.000000  1443.000000   1443.000000      1443.000000   
mean         929.880380     5.904186      2.243243        11.722800   
std          111.243312     0.926376      0.499390         1.349762   
min          299.093773     0.510500      2.000000         5.000000   
25%          913.485045     5.544500      2.000000        11.000000   
50%          937.488416     5.996000      2.000000        12.000000   
75%          977.519717     6.401700      2.000000        13.000000   
max         1154.630408     8.069200      6.000000        15.000000   

              TPSA  Rotatable bonds          IC50  
count  1443.000000      1443.000000   1443.000000  
mean    168.349473         9.802495   2138.860707  
std      19.032505         1.439658   3281.785689  
min      54.250000         3.000000      2.000000  
25%     161.890000         9.000000     55.000000  
50%     171.540000        10.000000    550.00000

In [5]:
import plotly.express as px

features = ['Molecular weight', 'LogP', "H-bond donor","H-bond acceptor","TPSA","Rotatable bonds", "IC50"]
features = df_features[features]

for column in features.columns:
    fig = px.box(features, x=column, color_discrete_sequence=['blue'])
    fig.update_layout(title=f'{column} Boxplot')
    fig.show()


In [9]:
df = df_features.drop_duplicates(subset=['SMILES'])
print(len(df2))


summary = df.describe()
print(summary)
summary.to_excel('../Results/summary.xlsx', index=False)


578
       Molecular weight        LogP  H-bond donor  H-bond acceptor  \
count        578.000000  578.000000    578.000000       578.000000   
mean         919.971656    5.807332      2.311419        11.593426   
std          123.892574    1.018012      0.561541         1.435774   
min          299.093773    0.510500      2.000000         5.000000   
25%          904.734815    5.394465      2.000000        11.000000   
50%          936.538432    5.945600      2.000000        12.000000   
75%          974.544157    6.389300      3.000000        12.000000   
max         1154.630408    8.069200      6.000000        15.000000   

             TPSA  Rotatable bonds          IC50  
count  578.000000       578.000000    578.000000  
mean   166.919446         9.806228   1668.984429  
std     21.137082         1.557838   3352.038966  
min     54.250000         3.000000      2.000000  
25%    158.650000         9.000000     55.000000  
50%    171.540000        10.000000    550.000000  
75%    1

In [10]:
import plotly.express as px

features_2 = ['Molecular weight', 'LogP', "H-bond donor","H-bond acceptor","TPSA","Rotatable bonds", "IC50"]
features_2 = df[features_2]

for column in features_2.columns:
    fig = px.box(features_2, x=column, color_discrete_sequence=['blue'])
    fig.update_layout(title=f'{column} Boxplot')
    fig.show()


In [11]:
import plotly.graph_objects as go
import numpy as np

x0 = features['IC50']
x1 = features_2['IC50']

fig = go.Figure()
# Use x instead of y argument for horizontal plot
fig.add_trace(go.Box(x=x0, name='All data'))
fig.add_trace(go.Box(x=x1, name='Without duplicates'))

fig.update_layout(title='IC50')

fig.show()

In [45]:
df.to_csv('../Data/NRAS_ligands_features.csv', index=False)